<a href="https://colab.research.google.com/github/deepaskori/JPMC-Data-for-Good/blob/main/DataForGoodFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from google.colab import files
uploaded = files.upload()


Saving NYC Schools -2021 Hackathon.xlsx to NYC Schools -2021 Hackathon.xlsx


In [ ]:
df1 = pd.read_excel("NYC Schools -2021 Hackathon.xlsx", sheet_name= '2016 School Explorer')
df1

In [ ]:
df2 = pd.read_excel("NYC Schools -2021 Hackathon.xlsx", sheet_name= 'Total - GenEd')
df2

In [ ]:
df3 = pd.read_excel("NYC Schools -2021 Hackathon.xlsx", sheet_name= 'GenEd - Parent %')
df3

In [ ]:
df4 = pd.read_excel("NYC Schools -2021 Hackathon.xlsx", sheet_name= 'GenEd - Student %')
df4

In [ ]:
df5 = pd.read_excel("NYC Schools -2021 Hackathon.xlsx", sheet_name= 'Teacher %')
df5

In [ ]:
df6 = pd.read_excel("NYC Schools -2021 Hackathon.xlsx", sheet_name= 'School Programs Counts')
df6

In [ ]:
df7 = pd.read_excel("NYC Schools -2021 Hackathon.xlsx", sheet_name= 'School Programs')
df7

In [ ]:
#correlating attendance with economic need index, school income estimate, rigorous instruction, collaborative teachers, supportive environment, effective school leadership, strong family-community ties, trust, ELA proficiency, Math proficiency
df1.head()
df = df[['Student Attendance Rate', 'Economic Need Index', 'School Income Estimate', 'Rigorous Instruction %', 'Collaborative Teachers %', 'Supportive Environment %', 'Effective School Leadership %', 'Strong Family-Community Ties %', 'Trust %', 'Average ELA Proficiency', 'Average Math Proficiency']]
# df.isna().any()
df.dropna()
df.corr()
#high correlation: ENI, SIE
#medium correlation: rigorous instruction, supportive environment, effective school leadership, strong fam-comm ties, avg ela proficiency
#low correlation: collaborative teachers, trust, average math prof

In [ ]:
#checking that DBN and Location Code are primary keys
df1[['School Name', 'Location Code']].head(10)


In [ ]:
df2[['School Name', 'DBN']].head(10)

In [ ]:
#joining df1 and df2 on primary key(DBN)
df1.rename(columns = {'Location Code':'DBN'}, inplace = True)
merged12 = pd.merge(df1[['DBN', 'Student Attendance Rate']], df2, on = 'DBN')
merged12.dropna()
merged12.corr()
#high correlation: total student response rate, total parent response rate, total teacher response rate
#medium correlation: collaborative teachers, effective school leadership, strong fam-comm ties, parent, teacher
#low correlation:trust

In [ ]:
#creating scatter plots for df1
from matplotlib import pyplot as plt
plt.scatter(df1['Economic Need Index'], np.log(df1['Student Attendance Rate']), alpha = .1)
plt.xlabel('Economic Need Index')
plt.ylabel('log of Student Attendance Rate')
plt.show()

In [ ]:
plt.scatter(df1['School Income Estimate'], np.log(df1['Student Attendance Rate']), alpha = .1)
plt.ylabel('School Income Estimate')
plt.xlabel('log of Student Attendance Rate')
plt.show()

In [ ]:
plt.scatter((merged12['Total Student Response Rate']), np.log(merged12['Student Attendance Rate']), alpha = .1)
plt.ylabel('Total Student Response Rate')
plt.xlabel('log of Student Attendance Rate')
plt.show()

In [ ]:
plt.scatter((merged12['Total Parent \nResponse Rate']), np.log(merged12['Student Attendance Rate']), alpha = .1)
plt.ylabel('Total Parent \nResponse Rate')
plt.xlabel('log of Student Attendance Rate')
plt.show()

In [ ]:
plt.scatter((merged12['Total Teacher Response Rate']), np.log(merged12['Student Attendance Rate']), alpha = .1)
plt.ylabel('Total Teacher Response Rate')
plt.xlabel('log of Student Attendance Rate')
plt.show()

In [ ]:
#grouping school data by zip code and finding correlation btwn attendance and specific programs

#creating zipcode column in df1
df1['Zip'] = df1['Address (Full)'].str[-6:]
df1['Zip'] = pd.to_numeric(df1.zipcode)

#grouping df1 by zipcode
df1_grouped = df1[['Student Attendance Rate', 'Zip']]
df1_grouped = df1_grouped.groupby(['Zip']).mean()
df1_grouped

#joining df6 and df1_grouped on 'zipcode'
merged16 = pd.merge(df1_grouped, df6, on = 'Zip')
merged16.corr()

,Zip,Student Attendance Rate,SUMMER_prog,SCHOOL_YEA_prog,WEEKENDS_prog,EVENINGS_prog,ELEMENTARY_prog,MIDDLE_SCH_prog,HIGH_SCHOO_prog,ACADEMICS_prog,SPORTS_PHY_prog,ARTS_CULTU_prog,Count_of_programs
Zip,1.000000,0.243670,-0.095024,-0.085243,-0.194370,-0.348096,-0.071544,-0.155459,-0.178704,-0.068497,-0.066123,-0.073533,-0.069640
Student Attendance Rate,0.243670,1.000000,-0.396858,-0.339737,-0.206951,-0.387550,-0.306027,-0.344693,-0.391637,-0.336530,-0.335674,-0.332522,-0.338425
SUMMER_prog,-0.095024,-0.396858,1.000000,0.871989,0.683616,0.821550,0.909714,0.891735,0.878235,0.913022,0.882942,0.901717,0.906781
SCHOOL_YEA_prog,-0.085243,-0.339737,0.871989,1.000000,0.606280,0.774422,0.935061,0.910857,0.905182,0.979936,0.964431,0.969119,0.988404
WEEKENDS_prog,-0.194370,-0.206951,0.683616,0.606280,1.000000,0.665151,0.659012,0.711037,0.640580,0.631804,0.562723,0.627726,0.622552
EVENINGS_prog,-0.348096,-0.387550,0.821550,0.774422,0.665151,1.000000,0.828740,0.860420,0.818555,0.799501,0.784634,0.802485,0.782768
ELEMENTARY_prog,-0.071544,-0.306027,0.909714,0.935061,0.659012,0.828740,1.000000,0.922481,0.826220,0.966769,0.956531,0.971267,0.955117
MIDDLE_SCH_prog,-0.155459,-0.344693,0.891735,0.910857,0.711037,0.860420,0.922481,1.000000,0.847243,0.939917,0.926781,0.941608,0.927593
HIGH_SCHOO_prog,-0.178704,-0.391637,0.878235,0.905182,0.640580,0.818555,0.826220,0.847243,1.000000,0.887098,0.853738,0.865009,0.905912
ACADEMICS_prog,-0.068497,-0.336530,0.913022,0.979936,0.631804,0.799501,0.966769,0.939917,0.887098,1.000000,0.988546,0.995501,0.994184


In [ ]:
#finding features regarding safety in df3
# df3.iloc[:, [66]] #At this school my child is safe.
# df3.iloc[:, [76]] #School Safety Agents promote a safe and respectful environment at this school.
#merge df3 and df1 on DBN

merged13 = pd.merge(df1[['DBN', 'Student Attendance Rate']], df3, on = 'DBN')
# merged13.head()
merged13.rename(columns = {"Unnamed: 66":'At this school my child is safe', "Unnamed: 76":"School Safety Agents promote a safe and respectful environment at this school."}, inplace = True)
merged13['At this school my child is safe'] = pd.to_numeric(merged13["At this school my child is safe"])
merged13["School Safety Agents promote a safe and respectful environment at this school."] = pd.to_numeric(merged13["School Safety Agents promote a safe and respectful environment at this school."])
merged13.iloc[:, [1, 67, 77]].corr()


,Student Attendance Rate,At this school my child is safe,School Safety Agents promote a safe and respectful environment at this school.
Student Attendance Rate,1.000000,0.022147,0.002250
At this school my child is safe,0.022147,1.000000,0.915829
School Safety Agents promote a safe and respectful environment at this school.,0.002250,0.915829,1.000000


In [ ]:
#features for safety df4
# df4.head()
df4.iloc[:, [143]] # 5i. School Safety Agents promote a safe and respectful environment at this school.
df4.iloc[:, [167]] #6f. In how many of your classes at this school do students feel it is important to come to school every day?
df4.iloc[:, [183]] #7a. I feel safe outside around this school.
df4.iloc[:, [187]] #7b. I feel safe traveling between home and this school.
df4.iloc[:, [191]] #7c. I feel safe in the hallways, bathrooms, locker rooms, and cafeteria of this school.
df4.iloc[:, [195]] #7d. I feel safe in my classes at this school.
merged14 = pd.merge(df1[['DBN', 'Student Attendance Rate']], df4, on = 'DBN')

merged14.rename(columns = {'Unnamed: 143': 'School Safety Agents promote a safe and respectful environment at this school.',
                           'Unnamed: 167':'In how many of your classes at this school do students feel it is important to come to school every day?', 
                           'Unnamed: 183': 'I feel safe outside around this school.',
                           'Unnamed: 187': 'I feel safe traveling between home and this school.', 
                           'Unnamed: 191': 'I feel safe in the hallways, bathrooms, locker rooms, and cafeteria of this school.', 
                           'Unnamed: 195': 'I feel safe in my classes at this school.'}, inplace = True)
merged14 = merged14.iloc[:, [1, 144, 168, 184, 188, 192, 196]]
merged14.apply(pd.to_numeric).corr()

,Student Attendance Rate,School Safety Agents promote a safe and respectful environment at this school.,In how many of your classes at this school do students feel it is important to come to school every day?,I feel safe outside around this school.,I feel safe traveling between home and this school.,"I feel safe in the hallways, bathrooms, locker rooms, and cafeteria of this school.",I feel safe in my classes at this school.
Student Attendance Rate,1.000000,0.137214,0.021754,0.221810,0.218222,0.215459,0.239152
School Safety Agents promote a safe and respectful environment at this school.,0.137214,1.000000,0.590694,0.794148,0.779496,0.854864,0.832313
In how many of your classes at this school do students feel it is important to come to school every day?,0.021754,0.590694,1.000000,0.547749,0.537196,0.518581,0.543182
I feel safe outside around this school.,0.221810,0.794148,0.547749,1.000000,0.933231,0.881609,0.870178
I feel safe traveling between home and this school.,0.218222,0.779496,0.537196,0.933231,1.000000,0.849244,0.861995
"I feel safe in the hallways, bathrooms, locker rooms, and cafeteria of this school.",0.215459,0.854864,0.518581,0.881609,0.849244,1.000000,0.920964
I feel safe in my classes at this school.,0.239152,0.832313,0.543182,0.870178,0.861995,0.920964,1.000000


In [ ]:
#features for safety df5
df5.iloc[:, [181]] #7g. School Safety Agents promote a safe and respectful environment at this school.
merged15 = pd.merge(df1[['DBN', 'Student Attendance Rate']], df5, on = 'DBN')
merged15 = merged15.iloc[:, [1, 182]]
merged15.rename(columns = {'Unnamed: 181': 'School Safety Agents promote a safe and respectful environment at this school.'}, inplace = True)
merged15
merged15.apply(pd.to_numeric).corr()

,Student Attendance Rate,School Safety Agents promote a safe and respectful environment at this school.
Student Attendance Rate,1.000000,0.061295
School Safety Agents promote a safe and respectful environment at this school.,0.061295,1.000000
